<h2>Table comparing different possible Cl2-to-CH4 ratios</h2>

In [1]:
# Some imports to stop some annoying yellow underlines
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
import pandas as pd
import importlib

# Very important line to run the setup notebook!!!
%run do_plot_setup.ipynb
import cl2_utilities as cl2_utils

# Extract some shared values that are used to facilitate plotting. Call 'display' on any of them to see their contents.
plot_default_properties = get_and_apply_plot_defaults() #Ignore yellow underline
line_default_properties = get_default_line_properties() #Ignore yellow underline
cl2_experiment_constants = get_cl2_experiment_constants() #Ignore yellow underline
(all_conversion_data,all_bypass_data) = get_experimental_data() #Ignore underline. These two dataframes have all the experimental results.

In [2]:
dummy_df = df = pd.DataFrame(np.random.randint(0,5,size=(5, 4)), columns=list('ABCD'))
display(dummy_df)

,A,B,C,D
0,2,0,0,1
1,3,1,2,4
2,4,4,1,4
3,4,4,1,2
4,0,0,4,2
